# Model scoring
Score all sequences in generated_seq


In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pickle


import os, sys

main_path = "../"

sys.path.append(main_path + "PGM/source/")
sys.path.append(main_path + "PGM/utilities/")
import utilities, Proteins_utils, sequence_logo, plots_utils
# be inside main directory while importing following


import os
import sys

# If running in an environment where __file__ is not defined
try:
    current_script_path = os.path.dirname(__file__)
except NameError:
    # If __file__ is not defined, fall back to the current working directory
    current_script_path = os.getcwd()

main_path = os.path.abspath(os.path.join(current_script_path, "../"))
os.chdir(main_path)
# list files in main directory
print(os.listdir(main_path))
if main_path not in sys.path:
    sys.path.append(main_path)
try:
    # import from main, not from current directory

    from global_variables import *
    from AB_RBM_EnergyModel import *
    from utils_evaluate_seq import *
except ImportError as e:
    raise ImportError(f"Failed to import one or more modules: {e}")

kd_vec=KD_VECTORS|NEW_KD_VECTORS
kd_ace2=ACE2_KD_VECTOR

# MAKE SURE UTILS EVALUATES HAS RIGHT KD VECTOR


e:\TRAJECTORIES_COVID\esmif\../PGM/source\numba_utilities.py:1124: NumbaPerformanceWarning: np.dot() is faster on contiguous arrays, called on (Array(float32, 2, 'F', False, aligned=True), Array(float32, 2, 'A', False, aligned=True))
  dmean_v_dw = np.dot(s1.T, V)
e:\TRAJECTORIES_COVID\esmif\../PGM/source\numba_utilities.py:961: NumbaPerformanceWarning: np.dot() is faster on contiguous arrays, called on (Array(float32, 1, 'A', False, aligned=True), Array(float32, 2, 'A', False, aligned=True))
  mean_V = np.dot(weights, V) / sum_weights
C:\Users\maria\AppData\Roaming\Python\Python312\site-packages\Bio\pairwise2.py:278: BiopythonDeprecationWarning: Bio.pairwise2 has been deprecated, and we intend to remove it in a future release of Biopython. As an alternative, please consider using Bio.Align.PairwiseAligner as a replacement, and contact the Biopython developers if you still need the Bio.pairwise2 module.
  warnings.warn(


['.git', '.gitignore', '6M0J.pdb', 'ab_improvement', 'AB_RBM_EnergyModel.py', 'ais_model.py', 'auc_comparison_nature.png', 'bash_scripts', 'covid_cases_data', 'data', 'esmif', 'exp_data', 'ext_plots', 'fields_l2_rbm_biaised_wt.ipynb', 'fields_RBM_Covid.data', 'fit_concentration', 'generated_sequences_ace2tensor(-9.3000)_betatensor(-0.4463).npy', 'generate_seqs', 'gisaid_data', 'global_variables.py', 'hugo_seq_analysis', 'indep_model.ipynb', 'indep_model_probas.npy', 'model_testing', 'new_RBM_Covid_best_test.data', 'obsolete', 'obsolete_structure', 'PGM', 'plot', 'Readme.md', 'roc_curve.png', 'roc_curves_publication.png', 'single_variants.fasta', 'Site_indep_model.py', 'test_rbm_biaised_wt.ipynb', 'test_wt_RBM_Covid.data', 'tmp.fasta', 'tmp_aligned.fasta', 'tmp_target.fasta', 'Trajectories_covid.code-workspace', 'ungrouped_gisaid', 'ungrouped_gisaid_spread_df_seq.fasta', 'utils_evaluate_seq.py', '__pycache__']
Loaded 29 KD vectors
Loaded 2 KD vectors


In [3]:
from Proteins_utils import *

In [4]:

ab_names=list(kd_vec.keys())
raw_concentrations = [-14 for _ in range(len(ab_names))]


model = ACE2_Beta_EnergyModel(
    raw_ace2=-9.36,
    raw_beta=np.log(0.74),
    raw_concentrations=raw_concentrations,
    kd_vectors=kd_vec,
)

In [10]:
wt_rbd=Proteins_utils.load_FASTA('data/rbd_wt.fasta')[:,BEGIN:-END]
print(wt_rbd.shape)
wt_score=-model(torch.tensor(wt_rbd)[0]).detach().numpy()
print(wt_score)


(1, 178)
1190.4817


In [9]:
seq=Proteins_utils.load_FASTA('esmif/generated_seq/generated_seq_Beta0.4.fasta')
score=-model(torch.tensor(seq)[0]).detach().numpy()
score

1162.262

In [18]:
import os
import torch
import pandas as pd
from Proteins_utils import load_FASTA

# Define input and output directories
input_dir = "esmif/generated_seq"
output_dir = "esmif/seq_scores_model"
os.makedirs(output_dir, exist_ok=True)

# Process all FASTA files in the input directory
for fasta_filename in os.listdir(input_dir):
    if fasta_filename.endswith(".fasta"):
        input_path = os.path.join(input_dir, fasta_filename)
        output_path = os.path.join(output_dir, f"model_lll_{fasta_filename.replace('.fasta', '.csv')}")
        
        # Load sequences from FASTA file
        seq=Proteins_utils.load_FASTA('esmif/generated_seq/'+fasta_filename)
        scores = [-model(torch.tensor(s)).detach().numpy() for s in seq]
        seqids = [f"{i}" for i in range(len(seq))]
        # Save scores to CSV
        df = pd.DataFrame({"seqid": seqids, "model_score": scores})
        df.to_csv(output_path, index=False)
        
        print(f"Processed {fasta_filename}")

print("All FASTA files processed.")


Processed generated_seq_Beta0.4.fasta
Processed generated_seq_Beta0.6.fasta
Processed generated_seq_Beta0.8.fasta
Processed generated_seq_Beta1.0.fasta
Processed generated_seq_Beta1.0_Ab_COV2-2130.fasta
Processed generated_seq_Beta1.0_Ab_COV2-2196.fasta
Processed generated_seq_Beta1.0_Ab_LY-CoV555.fasta
Processed generated_seq_Beta1.0_Ab_REGN10933.fasta
Processed generated_seq_Beta1.0_Ab_S2E12.fasta
Processed generated_seq_Beta1.0_Ab_S309.fasta
Processed generated_seq_Beta1.0_Ab_SA55.fasta
Processed generated_seq_Beta1.0_Ab_VIR7229.fasta
Processed generated_seq_Beta1.5.fasta
Processed generated_seq_Beta2.0.fasta
Processed generated_seq_Beta3.0.fasta
All FASTA files processed.
